In [1]:
import xml.etree.ElementTree as ET
import glob
import os
import json

In [2]:
def xml_to_yolo_bbox(bbox, w, h):
    # xmin, ymin, xmax, ymax
    x_center = min(((bbox[2] + bbox[0]) / 2) / w,1)
    y_center = min(((bbox[3] + bbox[1]) / 2) / h,1)
    width = min((bbox[2] - bbox[0]) / w,1)
    height = min((bbox[3] - bbox[1]) / h,1)
    return [x_center, y_center, width, height]
def yolo_to_xml_bbox(bbox, w, h):
    # x_center, y_center width heigth
    w_half_len = (bbox[2] * w) / 2
    h_half_len = (bbox[3] * h) / 2
    xmin = int((bbox[0] * w) - w_half_len)
    ymin = int((bbox[1] * h) - h_half_len)
    xmax = int((bbox[0] * w) + w_half_len)
    ymax = int((bbox[1] * h) + h_half_len)
    return [xmin, ymin, xmax, ymax]

In [3]:
classes = []
input_dir = "annotations/"
output_dir = "labels/"
image_dir = "images/"
os.mkdir(output_dir)

In [11]:
files = glob.glob(os.path.join(input_dir, '*.xml'))
for fil in files:
    basename = os.path.basename(fil)
    filename = os.path.splitext(basename)[0]
    # check if the label contains the corresponding image file
    if not os.path.exists(os.path.join(image_dir, f"{filename}.png")):
        print(f"{filename} image does not exist!")
        continue

    result = []

    # parse the content of the xml file
    tree = ET.parse(fil)
    root = tree.getroot()
    width = int(root.find("size").find("width").text)
    height = int(root.find("size").find("height").text)

    for obj in root.findall('object'):
        label = obj.find("name").text
        # check for new classes and append to list
        if label not in classes:
            classes.append(label)
        index = classes.index(label)
        pil_bbox = [int(x.text) for x in obj.find("bndbox")]
        yolo_bbox = xml_to_yolo_bbox(pil_bbox, width, height)
        # convert data to string
        bbox_string = " ".join([str(x) for x in yolo_bbox])
        result.append(f"{index} {bbox_string}")

    if result:
        # generate a YOLO format text file for each xml file
        with open(os.path.join(output_dir, f"{filename}.txt"), "w", encoding="utf-8") as f:
            f.write("\n".join(result))

# generate the classes file as reference

In [ ]:
with open('classes.txt', 'w', encoding='utf8') as f:
    f.write(json.dumps(classes))

# split data

In [14]:
J = 'kiwi_co__1_1'
J.rfind('_')
J[:10]

'kiwi_co__1'

In [15]:
img_dir = 'images/'
txt_dir = 'labels/'
#dir = 'drive/MyDrive/no_resize_no_augmentation/'
import os
import random
import shutil

nb_image = len(os.listdir(img_dir))
list_images = []
for i in os.listdir(img_dir):
  if i[i.find('__')+2:].find('_') == -1 or (i.find('logo') != -1 and i[i.find('__')+2:].count('_') == 1):
    list_images.append(i)
random.shuffle(list_images)

test_images = list_images[:int(len(list_images)/10)]
train_images = list_images[len(test_images):]
test_txt = [x[:-3]+ 'txt' for x in test_images]
train_txt = [x[:-3]+ 'txt' for x in train_images]
train_txt

['dollar_shave_club__logo_0.txt',
 'worlds_of_warships__logo_32.txt',
 'redbull__6.txt',
 'gfuel__logo_13.txt',
 'dollar_shave_club__logo_40.txt',
 'displate__logo_41.txt',
 'republic_of_gamers__14.txt',
 'microsoft__15.txt',
 'skillshare__logo_11.txt',
 'squarespace__logo_29.txt',
 'coca_cola__17.txt',
 'republic_of_gamers__logo_2.txt',
 'surfshark__logo_10.txt',
 'nordvpn__4.txt',
 'coca_cola__38.txt',
 'republic_of_gamers__logo_44.txt',
 'amazon__24.txt',
 'brilliant_org__logo_35.txt',
 'squarespace__logo_38.txt',
 'raid_shadow_legends__logo_15.txt',
 'uber_eats__34.txt',
 'ridge_wallet__logo_19.txt',
 'logitech__logo_23.txt',
 'uber_eats__logo_38.txt',
 'redbull__logo_9.txt',
 'state_of_survival__12.txt',
 'ridge_wallet__38.txt',
 'manscaped__logo_12.txt',
 'raid_shadow_legends__34.txt',
 'squarespace__logo_7.txt',
 'worlds_of_warships__10.txt',
 'worlds_of_warships__60.txt',
 'squarespace__logo_24.txt',
 'winamax__36.txt',
 'amazon__logo_14.txt',
 'skillshare__logo_36.txt',
 'skil

In [17]:
for image in os.listdir(img_dir):

  if image in ['train','test']:
    continue

  if image in test_images or image in train_images:
    image_root = image
  else:
    image_root = image[:image.rfind('_')]+ '.png'

  if image_root in test_images:
    shutil.move(img_dir + image, img_dir + 'test/' + image)
  if image_root in train_images:
    shutil.move(img_dir + image, img_dir + 'train/' + image)


In [16]:

for txt in os.listdir(txt_dir):
  if txt in ['train','test']:
    continue

  if txt in test_txt or txt in train_txt:
    txt_root = txt
  else:
    txt_root = txt[:txt.rfind('_')]+ '.txt'


  if txt_root in test_txt:
    shutil.move(txt_dir + txt, txt_dir + 'test/' + txt)
  if txt_root in train_txt:
    shutil.move(txt_dir + txt, txt_dir + 'train/' + txt)